In [ ]:
import mechanicalsoup
import bs4, re
import base64
from appscript import *

In [ ]:
browser = mechanicalsoup.Browser()

In [ ]:
def getSoup(url):
    page = browser.get(url)
    soup = bs4.BeautifulSoup(page.content, 'html.parser')
    return soup

In [ ]:
def getSubLinks(soup,tag="article"):
    links=[]
    for article in soup.find_all(tag):
        href=article.find('a')["href"]
        if linkFilter:
            lF=[de(l) for l in linkFilter]
            match="("+"|".join(lF)+")"
            m=re.search(match,href)
            if not m:
                links.append(article.find('a')["href"])
        else:
            links.append(article.find('a')["href"])
    return links

In [ ]:
def getTwitterSecurelyLink(url):
    link=url
    for a in getSoup(url).find_all("a"):
        href=a.get("href")
        if href and "url=https://openload.co" in href:
            link=href.split("url=")[-1]
    return link

In [ ]:
def getSecurelyLink(url):
    link=url
    for meta in getSoup(url).find_all("meta"):
        content=meta.get("content")
        if content and "://openload.co" in content:
            link=content
    if link==url:
        print "pb",link
        link=getTwitterSecurelyLink(url)
    elif "securely.link" in link:
        print "securely",link
        link=getSecurelyLink(link)
    return link

In [ ]:
def getOpenloadLinks(soup):
    links=[]
    for article in soup.find_all("div",class_="entry-content"):
        aSoup=article.find_all('a')
        for a in aSoup:
            link=a["href"]
            if "openload.co" in link:
                links.append(link)
            elif "openload.co" in a.text.lower() and "securely.link" in link:
                links.append(getSecurelyLink(link))
    return links

In [ ]:
def getNextPage(soup):
    nextPage=soup.find("a", class_="next page-numbers")["href"]
    return getSoup(nextPage)

In [ ]:
def en(s):
    return base64.b64encode(s)
def de(s):
    return base64.b64decode(s)
en("")

In [ ]:
linkFilter=["c2l0ZXJpcA==","bGVzYmlhbg=="]
urlBase="aHR0cDovL2Z1bGx4eHhtb3ZpZXMubmV0"
soup=getSoup(de(urlBase))
urls=getSubLinks(soup)

In [ ]:
getSoup(urls[0]).find("title")

In [ ]:
safari = app("Safari")
fileNames={}
numFin=20

In [ ]:
def pageLoop(urls,safariLoad=True,boolPrint=False):
    for url in urls:
        urlSoup=getSoup(url)
        for nLink,link in enumerate(getOpenloadLinks(urlSoup)):
            sLink=link.rsplit("/")[-1]
            if boolPrint:
                print urlSoup.title.text
                print sLink
                print link
            fileNames[sLink]=urlSoup.title.text+"-%d"%nLink
            if safariLoad:
                ok=raw_input("Press Enter to continue...")
                if ok.lower()!="n":
                    safari.windows.first.current_tab.URL.set(link)


In [ ]:
page=getSoup(de(urlBase))
for i in range(numFin):
    print "page %02d"%i
    print
    
    urls=getSubLinks(page)
    pageLoop(urls,safariLoad=False,boolPrint=False)
    page = getNextPage(page)


In [ ]:
fileNames

In [ ]:
l="http://securely.link/LgQ06"
getTwitterSecurelyLink(l)